#1) Introdução

Descrição: Este projeto visa fazer uma análise descritiva de dados de contratação do programa Minha Casa Minha Vida e Casa verde e Amarela.
São exploradas informações como:
- Estados com maiores contratações
- Empresas com maiores contratações, em âmbito nacional e por estado
- Tipologias de unidade habitacionais mais contratadas
- Entidades financeiras com maior envolvimento nas contratações


Ao final, é gerado um mapa de intensidade interativo visando facilitar a visualização das regiões do Brasil e sua intensidade de contratações.


Fonte dos dados (dados + dicionário dos dados): https://dados.gov.br/dados/conjuntos-dados/dados-de-programas-habitacionais---sistema-de-habitao---sishab

#2) Tratamento dos dados

In [1]:
!pip install geopy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [64]:
# Importando as bibliotecas

import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

# from pycep_correios import get_address_from_cep, WebService
from geopy.geocoders import Nominatim

%matplotlib inline

In [3]:
# Ocultando os avisos de output do Colab
import warnings
warnings.filterwarnings('ignore')

In [4]:
# Lendo o dataset

data=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Projetos pessoais/02. Análise exploratória de datasets/Programa Casa verde e Amarela/_contratacoes_pcmv_pcva.csv',encoding='latin-1',sep='|')

In [5]:
# Criando backup do dataset / criando o dataframe

df=data.copy()

In [ ]:
#Pré visualização do dataset

df.head()

In [ ]:
#Visualizando informações do dataframe

df.info()

In [9]:
#Convertendo alguns valores em números

df[['vlr_operacao','vlr_contrapartida','vlr_liberado']]=df[['vlr_operacao','vlr_contrapartida','vlr_liberado']].replace(',','.',regex=True).astype(float)

In [ ]:
# Visualização de dados missing no dataframe

df.isnull().sum()

In [ ]:
# Visualização de dados únicos no dataframe

df.nunique()

In [12]:
# Excluindo algumas colunas que não serão utilizadas

df=df.drop(['dte_assinatura_contrato','txt_nome_do_empreendimento','txt_apf_cod_empreendimento','txt_cnpj_proponente','txt_endereco','txt_cep'],axis=1)

In [13]:
# Ajustando a coluna 'num_ano_assinatura_contrato'

df['num_ano_assinatura_contrato']=df['num_ano_assinatura_contrato'].replace(',00','',regex=True).astype(int)

In [14]:
# Convertendo os valrres da coluna 'prc_obra' para numéricos
df['prc_obra']=df['prc_obra'].replace(',','.',regex=True).astype(float)

In [ ]:
# Avaliando os valores únicos das colunas txt_origem_contrato, txt_programa, txt_faixa_grupo_renda

for i in df[['txt_origem_contrato', 'txt_programa', 'txt_faixa_grupo_renda']]:
  print(list(df[i].unique()))

In [16]:
# Analisando as entradas da coluna 
# Esta coluna será excluída, pois já existe uma coluna informando qual o programa de financiamento

df['txt_faixa_grupo_renda'].unique()
df=df.drop('txt_faixa_grupo_renda',axis=1)

In [ ]:
# Quantidade de uniades entregues pelo Programa Minha Casa Minha Vida ( coma condição de opercentual de entrega ser 100%)
df.loc[(df['txt_programa']=='PMCMV')&(df['txt_situacao_obra']=='Obras Entregues') & (df['prc_obra']==100) ,'qtd_uh_entregues'].sum()

#3) Análise descritva

In [107]:
# Agrupando as informações por estado e mostrando pelo total de unidades contratadas e entregues
#Criando também uma coluna para mostrar o percentual de unidades entregues em relação às contratadas

df_uh=df[['txt_uf','qtd_uh_contratadas','qtd_uh_entregues']].groupby('txt_uf').sum().sort_values(by=['qtd_uh_contratadas'],ascending=False)
df_uh['Percentual']=round(df_uh['qtd_uh_entregues']/df_uh['qtd_uh_contratadas']*100,2)
df_uh


# Exibindo um gráfico
# df_uh=df_uh.sort_values(by=['qtd_uh_contratadas'],ascending=True)
# fig = px.bar(df_uh, x='qtd_uh_contratadas', y=df_uh.index.values ,orientation='h')
# fig.update_layout(xaxis_title="UH contratadas")
# fig.update_layout(title={'text':'Estados com mais contratações Minha Casa Minha Vida / Casa Verde e Amarela entre 2007 a 2022','x':0.5,'xanchor':'center','font':{'family':'Century Gothic','size':24}})
# fig.show()

,qtd_uh_contratadas,qtd_uh_entregues,Percentual
txt_uf,,,
SP,2416585,1690177,69.94
MG,1058708,833257,78.71
PR,742094,598675,80.67
RS,675187,515759,76.39
GO,581547,480125,82.56
RJ,563395,381333,67.68
BA,560381,416649,74.35
SC,387852,317376,81.83
PE,345490,246588,71.37


In [108]:
# Listando as empresas que mais contratam Unidades Habitacionais em âmbito nacional
df_empresas=df[['txt_razao_social_proponente','txt_uf','qtd_uh_contratadas']].groupby('txt_razao_social_proponente').sum().sort_values(by=['qtd_uh_contratadas'],ascending=False)

#Gerando um relatório em html
rel_empresas=df_empresas.head(20).to_html() #criando o html
with open('Relatório de empresas.html', 'w') as f:
    f.write(rel_empresas)

df_empresas

,qtd_uh_contratadas
txt_razao_social_proponente,
MRV ENGENHARIA E PARTICIPACOES S/A,370040
DIRECIONAL ENGENHARIA S.A.,112899
CONSTRUTORA TENDA S/A,110365
CURY CONSTRUTORA E INCORPORADORA S.A.,69135
PACAEMBU EMPREENDIMENTOS E CONSTRUCOES LTDA,53355
...,...
MUNICIPIO DE CAPELA DE SANTANA - RS,4
SOCIEDADE DE DESENVOLVIMENTO HUMANO SOCIAL E CULTURAL,4
ASSOCIACAO DOS PRODUTORES FAMILIARES DO CORREGO DO LAGE E RE,4


In [109]:
#Criando um gráfico para visualização das empresas que mais contratam
df_empresas=df_empresas.head(20).sort_values(by=['qtd_uh_contratadas'],ascending=True)
fig = px.bar(df_empresas, x='qtd_uh_contratadas', y=df_empresas.index.values ,orientation='h')
fig.update_layout(xaxis_title="UH contratadas")
fig.update_layout(title={'text':'Empresas com mais contratações Minha Casa Minha Vida / Casa Verde e Amarela entre 2007 e 2022','x':0.5,'xanchor':'center','font':{'family':'Century Gothic','size':24}})

fig.show()


In [ ]:
# Listando as empresas que mais contratam Unidades Habitacionais, separadas por Estados

# Cria uma lista com a UF de cada estado
uf=sorted((list(df['txt_uf'].unique())))  #extrai apenas os valores únicos e reordena em ordem alfabética

for i in uf:
  # Filtrando as empresas por estado
  df_emp_estado=df.loc[df['txt_uf']==i,['txt_razao_social_proponente','txt_uf','qtd_uh_contratadas','qtd_uh_entregues']].groupby('txt_razao_social_proponente').sum().sort_values(by=['qtd_uh_contratadas'],ascending=False)
  df_emp_estado['Percentual']=round(df_emp_estado['qtd_uh_entregues']/df_emp_estado['qtd_uh_contratadas']*100,2)

  # Exibindo o dataframe:
  # print(f'\n\n\n========================================================{i}=============================================================\n')
  # display(df_emp_estado.head(20))

  df_emp_estado=df_emp_estado.head(20).sort_values(by=['qtd_uh_contratadas'],ascending=True)
  fig = px.bar(df_emp_estado, x='qtd_uh_contratadas', y=df_emp_estado.index.values ,orientation='h')
  fig.update_layout(xaxis_title="UH contratadas")
  fig.update_layout(title={'text':f'Empresas com mais contratações entre 2007 e 2022 - {i}','x':0.5,'xanchor':'center','font':{'family':'Century Gothic','size':24}})

  fig.show()

  #Gerando um relatório em html:
  nome=(f'Top 20 empresas - {i}.html')  #concatena com o nome do estado
  rel_emp_estado=df_emp_estado.head(20).to_html() #criando o html
  with open(nome, 'w') as f:
    f.write(rel_emp_estado)

In [ ]:
# Listando tipologias mais contratadas, agrupadas por estado

# Criando o dataframe
df_tipo=df[['txt_uf','txt_tipologia','qtd_uh_contratadas']]

#dropando as linhas 'nao informada'
df_tipo=df_tipo.drop(df_tipo[df_tipo['txt_tipologia']=='Não Informada'].index)

# Fazendo o agrupamento
df_tipo=df_tipo.groupby(['txt_uf','txt_tipologia']).sum().sort_values(by=['txt_uf'],ascending=True)

#Resetando o index para consulta
df_tipo=df_tipo.reset_index()

#Plotando o gráfico
# fig = px.bar(df_tipo, x='txt_uf', y='qtd_uh_contratadas', color='txt_tipologia',barmode='group' ,title='Contratações por tipologia, agrupadas por estado')
fig = px.bar(df_tipo, x='txt_uf', y='qtd_uh_contratadas', color='txt_tipologia',barmode='overlay' ,opacity=1,title='Contratações por tipologia, agrupadas por estado')
fig.update_layout(title={'x':0.5,'xanchor':'center','font':{'family':'Century Gothic','size':24}})
fig.show()

In [ ]:
# Listando as instituições financeiras com mais envolvimento nas contratações, de acordo com o estado
# Criando o dataframe
df_banco=df[['txt_uf','txt_agente_financeiro','qtd_uh_contratadas']]

#dropando as linhas 'nao informada'
df_banco=df_banco.drop(df_banco[df_banco['txt_agente_financeiro']=='Não Informado'].index)

# Fazendo o agrupamento
df_banco=df_banco.groupby(['txt_uf','txt_agente_financeiro']).sum().sort_values(by=['txt_uf'],ascending=True)

#Resetando o index para consulta
df_banco=df_banco.reset_index()

#Plotando o gráfico
# fig = px.bar(df_tipo, x='txt_uf', y='qtd_uh_contratadas', color='txt_tipologia',barmode='group' ,title='Contratações por tipologia, agrupadas por estado')
fig = px.bar(df_banco, x='txt_uf', y='qtd_uh_contratadas', color='txt_agente_financeiro',barmode='overlay' ,opacity=1,title='Agentes financeiros com maior envolvimento nas contratações, agrupadas por estado')
fig.update_layout(title={'x':0.5,'xanchor':'center','font':{'family':'Century Gothic','size':24}})
fig.show()

In [ ]:
#Utilizando o CEP para extrair valores da latitude e longitude

df_cep=pd.DataFrame()

df_cep['Cidade']=data['txt_municipio'].apply(lambda x: x.strip()).replace(['ã','á','â','é','í','ô','õ','ú'],['a','a','a','e','i','o','o','u'],regex=True)+', '+data['txt_uf']
df_cep['UH contratadas']=data['qtd_uh_contratadas']
df_cep['UH entregues']=data['qtd_uh_entregues']
df_cep['End comp']=df_cep['Cidade']+', Brasil'

# df_cep.reset_index()

# >> Dropando para o dataframe ficar menor
# df_cep=df_cep.loc[0:500,:]

# #Agrupando os valores por cidade
df_cep=df_cep.groupby('End comp').sum()

# #Resentando o index. Após o agrupamento, a coluna usada como parâmetro se torna o index
# #É necessário resetar essa condição, para que a coluna esteja disponível paa consulta.
df_cep=df_cep.reset_index()

from geopy.extra.rate_limiter import RateLimiter

#Criando o locator do Nomoinim
locator = Nominatim(user_agent="MyGeocoder")
geocode=RateLimiter(locator.geocode,min_delay_seconds=1)
df_cep['coordenadas']=df_cep['End comp'].apply(geocode)

#Dropando as linhas que não tiveram retorno
d_cep=df_cep.dropna(inplace=True)

#Cria uma coluna com tuplas que representam as coordenadas x, y, z
df_cep['point'] = df_cep['coordenadas'].apply(lambda loc: tuple(loc.point) if loc else None)

#Extraindo o primeiro elemento da tupla (que é a coordenada x)
df_cep['Latitude']=df_cep['point'].apply(lambda x: x[0])

#Extraindo o segundo elemento da tupla (que é a coordenada x)
df_cep['Longitude']=df_cep['point'].apply(lambda x: x[1])

#Dropando as colunas de 'coordenadas' e 'point'
df_cep=df_cep.drop(['coordenadas','point'],axis=1)

df_cep.to_excel('Relação Endereço-Coordenadas.xlsx', index=False)

In [112]:
# Criando um mapa de visualização dos dados

import plotly.graph_objs as go
from IPython.display import HTML
from google.colab import files

# Dados de coordenadas
lat = df_cep['Latitude']
lon = df_cep['Longitude']
intensidade=df_cep['UH contratadas']

# Cria um mapa de calor de densidade do Mapbox
fig = go.Figure(go.Densitymapbox(lat=lat, lon=lon,z=intensidade,radius=10,zmin=1,zmax=10))
# fig = go.Figure(go.Densitymapbox(lat=lat, lon=lon,z=intensidade,radius=10,colorscale='Jet',zmin=0,zmax=10))
# zmin e zmax são a escala de intensidade das cores

# Define as configurações do layout do Mapbox
fig.update_layout(mapbox_style="stamen-terrain", mapbox_center_lon=-50, mapbox_center_lat=-15, mapbox_zoom=3.5)

# Gera título para o gráfico
fig.update_layout(title={'text':'Contratações Minha Casa Minha Vida / Casa Verde e Amarela entre 2007 e 2022','x':0.5,'xanchor':'center','font':{'family':'Century Gothic','size':24}})

# Atualiza as configurações do traço para mostrar a intensidade
fig.update_traces(hovertemplate="Contratações na cidade: %{z:.2f}<extra></extra>")

# Gera o arquivo HTML com o gráfico << funcionando
fig.write_html('mapa.html')

# Exibe o gráfico no notebook, em formato html >> funcionando
# HTML(filename='mapa.html')

#Faz o download do mapa.
files.download('mapa.html')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>